# Catalog inspection

Perform some basic verification on the datasets.

- confirm the number of nulls (NaNs) in the dataset is within expectations
- for fields with predictable limits, confirm min/max values in dataset

In [1]:
!pip install hats --quiet

In [1]:
%env DRP_VERSION=w_2025_07

env: DRP_VERSION=w_2025_07


In [2]:
import os
import hats
import numpy as np
import pandas as pd
import pyarrow.parquet as pq

from pathlib import Path

INFO:numexpr.utils:Note: detected 128 virtual cores but NumExpr set to maximum of 64, check "NUMEXPR_MAX_THREADS" environment variable.
INFO:numexpr.utils:Note: NumExpr detected 128 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 16.
INFO:numexpr.utils:NumExpr defaulting to 16 threads.


In [3]:
DRP_VERSION = os.environ["DRP_VERSION"]
print(f"DRP_VERSION: {DRP_VERSION}")
base_output_dir = Path(f"/sdf/data/rubin/shared/lsdb_commissioning")
raw_dir = base_output_dir / "raw" / DRP_VERSION
hats_dir = base_output_dir / "hats" / DRP_VERSION

DRP_VERSION: w_2025_07


In [4]:
fields_dir = raw_dir / "field_sizes"
fields_dir.mkdir(parents=True, exist_ok=True)

In [48]:
fresh_catalogs = [
    "diaObject",
    "diaSource",
    "diaForcedSource",
    "object",
    "source",
    "forcedSource",
]

for catalog_name in fresh_catalogs[4:5]:
    cat = hats.read_hats(hats_dir / catalog_name)
    print(catalog_name)
    print("  num partitions:", len(cat.get_healpix_pixels()))
    print("  num rows:", cat.catalog_info.total_rows)
    stats = cat.aggregate_column_statistics()
    print("  num columns:", len(stats))
    ## Remove columns with "Mag" as these are created for HATS
    stats = stats.iloc[~stats.index.str.contains("Mag")]
    np_null_count = stats["null_count"].to_numpy(copy=False, dtype=np.int64)
    with_null_index = np.where(np_null_count >0)
    with_nulls = stats.iloc[with_null_index]
    
    if len(with_nulls):
        print(f"  columns with nulls: {len(with_nulls)}")
        with_nulls = with_nulls[["null_count"]]
        with_nulls["percent"] = [null_count/cat.catalog_info.total_rows*100 for null_count in with_nulls["null_count"].to_numpy(copy=False, dtype=np.int64)]
        with_nulls = with_nulls.sort_values(by='percent', ascending=False)
        print(with_nulls)
    else:
        print("  columns with nulls: 0")
    

source
  num partitions: 117
  num rows: 43826352
  num columns: 169
  columns with nulls: 44
               null_count    percent
column_names                        
variance_value   11125405  25.385195
ixxDebiasedPSF    3423880   7.812377
...                   ...        ...
ixxPSF                  1   0.000002
iyyPSF                  1   0.000002

[44 rows x 2 columns]


In [49]:
import lsdb

cat = lsdb.read_hats(hats_dir / catalog_name)
cat

,coord_ra,coord_dec,parentSourceId,x,y,xErr,yErr,ra,dec,decl,raErr,decErr,ra_dec_Cov,calibFlux,calibFluxErr,ap03Flux,ap03FluxErr,ap03Flux_flag,ap06Flux,ap06FluxErr,ap06Flux_flag,ap09Flux,ap09FluxErr,ap09Flux_flag,ap12Flux,ap12FluxErr,ap12Flux_flag,ap17Flux,ap17FluxErr,ap17Flux_flag,ap25Flux,ap25FluxErr,ap25Flux_flag,ap35Flux,ap35FluxErr,ap35Flux_flag,ap50Flux,ap50FluxErr,ap50Flux_flag,ap70Flux,ap70FluxErr,ap70Flux_flag,sky,skyErr,psfFlux,psfFluxErr,ixx,iyy,ixy,ixxPSF,iyyPSF,ixyPSF,ixxDebiasedPSF,iyyDebiasedPSF,ixyDebiasedPSF,gaussianFlux,gaussianFluxErr,extendedness,sizeExtendedness,localPhotoCalib,localPhotoCalib_flag,localPhotoCalibErr,localWcs_flag,localWcs_CDMatrix_2_1,localWcs_CDMatrix_1_1,localWcs_CDMatrix_1_2,localWcs_CDMatrix_2_2,blendedness_abs,blendedness_flag,blendedness_flag_noCentroid,blendedness_flag_noShape,apFlux_12_0_flag,apFlux_12_0_flag_apertureTruncated,apFlux_12_0_flag_sincCoeffsTruncated,apFlux_12_0_instFlux,apFlux_12_0_instFluxErr,apFlux_17_0_flag,apFlux_17_0_instFlux,apFlux_17_0_instFluxErr,apFlux_35_0_flag,apFlux_35_0_instFlux,apFlux_35_0_instFluxErr,apFlux_50_0_flag,apFlux_50_0_instFlux,apFlux_50_0_instFluxErr,normCompTophatFlux_flag,normCompTophatFlux_instFlux,normCompTophatFlux_instFluxErr,extendedness_flag,sizeExtendedness_flag,footprintArea_value,invalidPsfFlag,jacobian_flag,jacobian_value,localBackground_instFlux,localBackground_instFluxErr,localBackground_flag,localBackground_flag_noGoodPixels,localBackground_flag_noPsf,pixelFlags_bad,pixelFlags_cr,pixelFlags_crCenter,pixelFlags_edge,pixelFlags_interpolated,pixelFlags_interpolatedCenter,pixelFlags_offimage,pixelFlags_saturated,pixelFlags_saturatedCenter,pixelFlags_suspect,pixelFlags_suspectCenter,psfFlux_apCorr,psfFlux_apCorrErr,psfFlux_area,psfFlux_flag,psfFlux_flag_apCorr,psfFlux_flag_edge,psfFlux_flag_noGoodPixels,gaussianFlux_flag,centroid_flag,centroid_flag_almostNoSecondDerivative,centroid_flag_badError,centroid_flag_edge,centroid_flag_noSecondDerivative,centroid_flag_notAtMaximum,centroid_flag_resetToPeak,variance_flag,variance_flag_emptyFootprint,variance_value,calib_astrometry_used,calib_photometry_reserved,calib_photometry_used,calib_psf_candidate,calib_psf_reserved,calib_psf_used,deblend_deblendedAsPsf,deblend_hasStrayFlux,deblend_masked,deblend_nChild,deblend_parentTooBig,deblend_patchedTemplate,deblend_rampedTemplate,deblend_skipped,deblend_tooManyPeaks,hsmPsfMoments_flag,hsmPsfMoments_flag_no_pixels,hsmPsfMoments_flag_not_contained,hsmPsfMoments_flag_parent_source,iDebiasedPSF_flag,iDebiasedPSF_flag_no_pixels,iDebiasedPSF_flag_not_contained,iDebiasedPSF_flag_parent_source,iDebiasedPSF_flag_galsim,iDebiasedPSF_flag_edge,hsmShapeRegauss_flag,hsmShapeRegauss_flag_galsim,hsmShapeRegauss_flag_no_pixels,hsmShapeRegauss_flag_not_contained,hsmShapeRegauss_flag_parent_source,sky_source,detect_isPrimary,visit,detector,band,physical_filter,sourceId,day_obs,psfMag,psfMagErr,midpointMjdTai,Norder,Dir,Npix
npartitions=117,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
"Order: 5, Pixel: 32",double[pyarrow],double[pyarrow],int64[pyarrow],double[pyarrow],double[pyarrow],float[pyarrow],float[pyarrow],double[pyarrow],double[pyarrow],double[pyarrow],float[pyarrow],float[pyarrow],float[pyarrow],double[pyarrow],double[pyarrow],double[pyarrow],double[pyarrow],bool[pyarrow],double[pyarrow],double[pyarrow],bool[pyarrow],double[pyarrow],double[pyarrow],bool[pyarrow],double[pyarrow],double[pyarrow],bool[pyarrow],double[pyarrow],double[pyarrow],bool[pyarrow],double[pyarrow],double[pyarrow],bool[pyarrow],double[pyarrow],double[pyarrow],bool[pyarrow],double[pyarrow],double[pyarrow],bool[pyarrow],double[pyarrow],double[pyarrow],bool[pyarrow],double[pyarrow],double[pyarrow],double[pyarrow],double[pyarrow],double[pyarrow],double[pyarrow],double[pyarrow],double[pyarrow],double[pyarrow],double[pyarrow],double[pyarrow],double[pyarrow],double[py

In [7]:
# Define the six fields with their coordinates
fields = {
    "ECDFS": (53.13, -28.10),  # Extended Chandra Deep Field South
    "EDFS": (59.10, -48.73),  # Euclid Deep Field South
    "Rubin_SV_38_7": (37.86, 6.98),  # Low Ecliptic Latitude Field
    "Rubin_SV_95_-25": (95.00, -25.00),  # Low Galactic Latitude Field
    "47_Tuc": (6.02, -72.08),  # 47 Tuc Globular Cluster
    "Fornax_dSph": (40.00, -34.45)  # Fornax Dwarf Spheroidal Galaxy
}

# Define the radius for selecting sources
selection_radius = 2.0  # 2-degree radius

In [54]:
from tqdm import tqdm

In [ ]:
# Define the six fields with their coordinates
fields = {
    "ECDFS": (53.13, -28.10),  # Extended Chandra Deep Field South
    "EDFS": (59.10, -48.73),  # Euclid Deep Field South
    "Rubin_SV_38_7": (37.86, 6.98),  # Low Ecliptic Latitude Field
    "Rubin_SV_95_-25": (95.00, -25.00),  # Low Galactic Latitude Field
    "47_Tuc": (6.02, -72.08),  # 47 Tuc Globular Cluster
    "Fornax_dSph": (40.00, -34.45)  # Fornax Dwarf Spheroidal Galaxy
}

# Define bands
bands = ["u", "g", "r", "i", "z", "y"]

# Define the radius for selecting sources (2 degrees converted to arcsec)
selection_radius_arcsec = 2 * 3600

# Function to compute statistics
def get_stats(df):
    stats = {}
    for band in bands:
        mask = df["band"] == band  # Filter by band
        if mask.sum() > 0:  # Ensure there are sources in this band
            stats[f"mean_x_{band}"] = np.mean(df.loc[mask, "x"])
            stats[f"mean_y_{band}"] = np.mean(df.loc[mask, "y"])
            stats[f"mean_psfFlux_{band}"] = np.mean(df.loc[mask, "psfFlux"])
            stats[f"len_{band}"] = len(df.loc[mask, "x"])
    return pd.DataFrame([stats])  # Convert to DataFrame

# Dictionary to store results
all_results = {}

# Loop through each field and perform cone search + computation
for field_name, (ra, dec) in fields.items():
    # Perform cone search for the given field
    field_cat = cat.cone_search(ra=ra, dec=dec, radius_arcsec=selection_radius_arcsec)
    
    # Compute statistics
    result = field_cat.map_partitions(
        get_stats,
        meta={f"mean_x_{band}": "f8" for band in bands} |
             {f"mean_y_{band}": "f8" for band in bands} |
             {f"mean_psfFlux_{band}": "f8" for band in bands} |
             {f"len_{band}": "i8" for band in bands}
    ).compute()

    # Compute weighted sum for each band separately
    weighted_means = {}
    for band in bands:
        if f"len_{band}" in result.columns and result[f"len_{band}"].sum() > 0:
            for col in [f"mean_x_{band}", f"mean_y_{band}", f"mean_psfFlux_{band}"]:
                if col in result.columns:
                    weighted_means[col] = np.sum(result[col] * result[f"len_{band}"]) / np.sum(result[f"len_{band}"])
    
    # Store the weighted means for this field
    all_results[field_name] = weighted_means

# Convert to DataFrame for better visualization
weighted_means_df = pd.DataFrame.from_dict(all_results, orient="index")


ValueError: The columns in the computed data do not match the columns in the provided metadata.
Order of columns does not match.
Actual:   ['mean_x_u', 'mean_y_u', 'mean_psfFlux_u', 'len_u', 'mean_x_g', 'mean_y_g', 'mean_psfFlux_g', 'len_g', 'mean_x_r', 'mean_y_r', 'mean_psfFlux_r', 'len_r', 'mean_x_i', 'mean_y_i', 'mean_psfFlux_i', 'len_i', 'mean_x_z', 'mean_y_z', 'mean_psfFlux_z', 'len_z', 'mean_x_y', 'mean_y_y', 'mean_psfFlux_y', 'len_y']
Expected: ['mean_x_u', 'mean_x_g', 'mean_x_r', 'mean_x_i', 'mean_x_z', 'mean_x_y', 'mean_y_u', 'mean_y_g', 'mean_y_r', 'mean_y_i', 'mean_y_z', 'mean_y_y', 'mean_psfFlux_u', 'mean_psfFlux_g', 'mean_psfFlux_r', 'mean_psfFlux_i', 'mean_psfFlux_z', 'mean_psfFlux_y', 'len_u', 'len_g', 'len_r', 'len_i', 'len_z', 'len_y']

In [56]:
weighted_means_df

,mean_x,mean_y,mean_psfFlux
ECDFS,2041.623842,2002.410963,32580.409744
EDFS,2042.635729,2006.498950,44595.771184
Rubin_SV_38_7,2040.680649,1996.240286,37421.390024
Rubin_SV_95_-25,2042.054492,2000.687755,107398.779519
47_Tuc,1975.995474,2004.174380,43525.070738
Fornax_dSph,2005.277416,1974.093942,9505.623190


In [40]:
weighted_means_df

,Weighted Mean
mean_x,2041.623842
mean_y,2002.410963
mean_psfFlux,32580.409744


In [37]:
result["mean_x"]

0    2683.935465
0    2439.995605
        ...     
0    2306.930765
0    2197.410974
Name: mean_x, Length: 41, dtype: float64